# Reinforcement Learning for Trading Strategies - DQL

### 1.0 Import Libraries

In [8]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam
import math
import numpy as np
import random
from collections import deque

### 2.0 Creating Agent

In [45]:
class Agent:
    def __init__(self,state_size,is_eval=False,model_name=""):
        self.state_size = state_size # normalized previous day
        self.action_size = 3 # sit,buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.model = load_model(model_name) if is_eval else self._model()
    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))
        return model
    def act(self, state):
        if not self.is_eval and random.random()<= self.epsilon:
            return random.randrange(self.action_size)
        options = self.model.predict(state)
        return np.argmax(options[0])
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

### 3.0 Define Basic Function

In [46]:
def formatPrice(n):
    return("-Rs." if n<0 else "Rs.")+"{0:.2f}".format(abs(n))
def getStockDataVec(key):
    vec = []
    lines = open(key+".csv","r").read().splitlines()
    for line in lines[1:]:
        #print(line)
        #print(float(line.split(",")[4]))
        vec.append(float(line.split(",")[4]))
        #print(vec)
    return vec 
def sigmoid(x):
    return 1/(1+math.exp(-x))
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])

In [47]:
t=0
n=10
d = t - n + 1
block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
res = []
for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
print(res)

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


### 3.0 Training the Agent

In [48]:
import sys
stock_name = input("Enter stock_name, window_size, Episode_count")
window_size = input()
episode_count = input()
stock_name = str(stock_name)
window_size = int(window_size)
episode_count = int(episode_count)
agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32
for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)
    total_profit = 0
    agent.inventory = []
    for t in range(l):
        action = agent.act(state)
        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0
        if action == 1: # buy
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))
        elif action == 2 and len(agent.inventory) > 0: # sell
            bought_price = window_size_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
    if e % 10 == 0:
        agent.model.save(str(e))

Enter stock_name, window_size, Episode_count7043.KL
10
1
Episode 0/1
Buy: Rs.0.40
Sell: Rs.0.39 | Profit: -Rs.0.01
Buy: Rs.0.49
Sell: Rs.0.49 | Profit: Rs.0.00
Buy: Rs.0.49
Buy: Rs.0.49
Sell: Rs.0.49 | Profit: Rs.0.00
Sell: Rs.0.49 | Profit: Rs.0.00
Buy: Rs.0.49
Buy: Rs.0.49
Sell: Rs.0.49 | Profit: Rs.0.00
Buy: Rs.0.49
Sell: Rs.0.49 | Profit: Rs.0.00
Buy: Rs.0.49
Sell: Rs.0.49 | Profit: Rs.0.00
Buy: Rs.0.49
Sell: Rs.0.49 | Profit: Rs.0.00
Sell: Rs.0.49 | Profit: Rs.0.00
Buy: Rs.0.49
Buy: Rs.0.49
[[0.24119173 0.11570881 0.20879751]]
2
[[0.29189643 0.15817364 0.26056948]]
2
[[0.34309652 0.20228672 0.31372598]]
0
[[0.35150442 0.22277637 0.3332692 ]]
2
[[0.36190248 0.24191907 0.3526394 ]]
2
[[0.36346844 0.25058267 0.36121756]]
2
[[0.34871137 0.24342854 0.35087058]]
1
[[0.35832354 0.26569757 0.3688389 ]]
0
[[0.3601954  0.27950582 0.37784904]]
0
[[0.35767803 0.2883991  0.38166463]]
0
[[0.35719374 0.30009186 0.3876104 ]]
0
[[0.3668505  0.31459188 0.40011716]]
2
[[0.37410405 0.32579076 0.40942

[[0.02580999 0.02592517 0.02522487]]
0
[[0.02577525 0.0256876  0.02507436]]
1
[[0.02576463 0.02539822 0.02490135]]
0
[[0.02557309 0.02515942 0.02469901]]
2
[[0.02538579 0.02493135 0.02449471]]
1
[[0.02523104 0.02467431 0.02428498]]
2
[[0.02508007 0.02443252 0.02407873]]
1
[[0.02495451 0.02417604 0.02387365]]
2
[[0.02483523 0.02393935 0.02367955]]
1
[[0.02473373 0.02370386 0.02349356]]
2
[[0.02464231 0.02349146 0.02332591]]
2
[[0.02456293 0.0233025  0.02317927]]
0
[[0.02431432 0.02315387 0.02300213]]
1
[[0.02409107 0.02301623 0.02284057]]
1
[[0.02432276 0.02307401 0.02300129]]
0
[[0.02307643 0.02284772 0.02236958]]
0
[[0.02265354 0.02272769 0.02215593]]
2
[[0.02152752 0.02278211 0.02177025]]
0
[[0.02140962 0.02213958 0.02128962]]
1
[[0.02039989 0.02283016 0.02141452]]
1
Buy: Rs.0.42
[[0.02214554 0.02308878 0.02235678]]
0
[[0.02164938 0.02306896 0.02218052]]
0
[[0.02055564 0.02218726 0.02113476]]
0
[[0.01827573 0.02227078 0.02029695]]
0
[[0.01883429 0.02203302 0.02035154]]
2
[[0.02147845

[[0.16042413 0.14356363 0.15942676]]
2
[[0.16180316 0.14593358 0.16217469]]
1
[[0.163826   0.15074988 0.16660361]]
0
[[0.16303687 0.1555073  0.16994806]]
1
[[0.16063079 0.16088594 0.17293622]]
0
[[0.16005155 0.16523644 0.17584662]]
2
[[0.15902197 0.16798426 0.17718992]]
1
[[0.15803312 0.1705621  0.17843746]]
2
[[0.15660259 0.1717698  0.17829998]]
1
[[0.15562306 0.1721983  0.17788579]]
2
[[0.15420963 0.17156827 0.1763301 ]]
1
[[0.15345517 0.16996893 0.17450318]]
2
[[0.15227708 0.16763273 0.17178997]]
2
[[0.15075827 0.16468716 0.16834785]]
0
[[0.1528835  0.16120185 0.16590096]]
1
[[0.15514052 0.15731928 0.16340746]]
1
[[0.15715067 0.15330334 0.16085081]]
0
[[0.15707523 0.15019812 0.15824035]]
0
[[0.15455994 0.14781325 0.15533006]]
2
[[0.15225673 0.14594536 0.15270486]]
0
[[0.14721751 0.14413862 0.14920567]]
1
[[0.14240962 0.14373323 0.14681111]]
1
[[0.13909924 0.14164159 0.14396732]]
1
[[0.13542229 0.14048384 0.1416916 ]]
2
[[0.1324743  0.13813177 0.13873334]]
2
[[0.13222115 0.13823816 0

[[0.19803071 0.19135068 0.20323502]]
2
[[0.19889624 0.19446225 0.20718393]]
2
[[0.19923484 0.19631447 0.20972219]]
2
[[0.19993296 0.19800043 0.21198295]]
0
[[0.2019379  0.19988076 0.21477172]]
1
[[0.20074555 0.2003214  0.21514133]]
0
[[0.20281346 0.20170751 0.21733038]]
2
[[0.2034275  0.20222142 0.21806405]]
1
[[0.20384315 0.2033554  0.21898906]]
2
[[0.20321631 0.20379253 0.21878225]]
1
[[0.2025378  0.20473967 0.21881223]]
2
[[0.2009275  0.2050075  0.21779983]]
1
[[0.19942477 0.20550954 0.21699008]]
2
[[0.19708888 0.20538232 0.21523723]]
2
[[0.19402175 0.20469594 0.21265452]]
0
[[0.19390246 0.20384888 0.21105418]]
1
[[0.19388804 0.20262899 0.20944162]]
1
[[0.19367135 0.2008377  0.20755461]]
0
[[0.19450939 0.1990831  0.20610668]]
0
[[0.19536717 0.19722442 0.20464495]]
2
[[0.1962535  0.19565375 0.20324524]]
0
[[0.19533141 0.1935032  0.2008934 ]]
1
[[0.19437031 0.19284384 0.19961445]]
1
[[0.19439588 0.19048126 0.19775431]]
1
[[0.19369544 0.18934134 0.19648324]]
2
[[0.19322364 0.18692778 0

[[0.23334815 0.22665413 0.24104781]]
2
[[0.2340812  0.22952211 0.24472193]]
2
[[0.2345864  0.230775   0.24685341]]
2
[[0.23562242 0.23172471 0.24865744]]
0
[[0.23736009 0.23308334 0.25089756]]
1
[[0.23598568 0.23304209 0.25074682]]
0
[[0.23742068 0.23423381 0.25243938]]
0
[[0.23931676 0.23507372 0.2539821 ]]
2
[[0.23974492 0.23507512 0.25408086]]
1
[[0.2400038  0.23605722 0.2545448 ]]
1
[[0.24012129 0.23784275 0.2553067 ]]
0
[[0.24105406 0.23965348 0.25638017]]
1
[[0.24256822 0.24155948 0.25765678]]
0
Buy: Rs.0.48
[[0.24355611 0.24327317 0.25857374]]
2
[[0.2436469  0.24412018 0.25833464]]
0
[[0.24437347 0.24484842 0.25830156]]
1
[[0.24501778 0.24559183 0.25830454]]
1
[[0.24573831 0.24578702 0.2580714 ]]
0
[[0.24492195 0.24635246 0.2575453 ]]
0
[[0.24478869 0.24615866 0.25688803]]
2
[[0.24318793 0.24643178 0.2557057 ]]
0
[[0.2416638  0.24546312 0.25385267]]
1
[[0.2400893  0.24572772 0.252962  ]]
1
[[0.24019256 0.2434689  0.25137195]]
1
[[0.23936155 0.24218854 0.25018704]]
2
[[0.23788226

...

[[0.07822101 0.07432848 0.08240168]]
2
[[0.0784381  0.07485887 0.08223908]]
2
[[0.07863358 0.07533645 0.08204064]]
2
[[0.0788096  0.07576649 0.08181001]]
0
[[0.0787495  0.07615372 0.08160234]]
2
[[0.07869538 0.0765024  0.08136365]]
1
[[0.07864664 0.07691447 0.08114871]]
2
[[0.07860276 0.07728553 0.08090372]]
1
[[0.07856325 0.07756665 0.08068312]]
2
[[0.07852767 0.07781978 0.080465  ]]
2
[[0.07849564 0.07804772 0.0802175 ]]
1
[[0.07846679 0.07802438 0.07999463]]
1
[[0.07844082 0.07775272 0.07979396]]
2
[[0.07841743 0.07750811 0.08037242]]
2
[[0.07839637 0.07728785 0.0814731 ]]
1
[[0.0783774  0.07710332 0.0824642 ]]
2
[[0.07836033 0.07693717 0.0833673 ]]
2
[[0.07834495 0.07678755 0.08412969]]
2
[[0.07833111 0.07665283 0.08476491]]
2
[[0.07831864 0.07653152 0.08528522]]
2
[[0.07830741 0.07642229 0.08570174]]
0
[[0.07891322 0.07632393 0.0860768 ]]
2
[[0.07945872 0.07623536 0.08636203]]
2
[[0.07994992 0.07615561 0.08656617]]
2
[[0.08039221 0.0760838  0.08669718]]
2
[[0.08079048 0.07601914 0

[[0.09306626 0.0912578  0.09846057]]
2
[[0.0930806  0.09156259 0.09820302]]
2
[[0.09309351 0.09183704 0.09791958]]
0
[[0.09309028 0.09208417 0.09766436]]
2
[[0.09308738 0.0923067  0.09738327]]
1
[[0.09308476 0.09253055 0.09713016]]
2
[[0.09308241 0.09273212 0.09685119]]
1
[[0.09308028 0.09283158 0.0966    ]]
2
[[0.09307837 0.09292115 0.09634928]]
2
[[0.09307665 0.09300179 0.09607282]]
1
[[0.0930751  0.09287802 0.09582388]]
1
[[0.09307371 0.09255816 0.09559973]]
2
[[0.09307245 0.09227014 0.09602458]]
2
[[0.09307132 0.0920108  0.09689017]]
1
[[0.0930703  0.09178121 0.0976696 ]]
2
[[0.09306938 0.09157449 0.09837263]]
2
[[0.09306855 0.09138834 0.09895524]]
2
[[0.09306781 0.09122073 0.09942912]]
2
[[0.09306714 0.0910698  0.09980484]]
2
[[0.09306654 0.0909339  0.10009197]]
2
[[0.09306599 0.09081152 0.10029918]]
2
[[0.0930655  0.09070133 0.1004343 ]]
2
[[0.09306506 0.09060211 0.10050444]]
2
[[0.09306467 0.09051276 0.10051601]]
2
[[0.09306431 0.09043231 0.10047482]]
2
[[0.09306399 0.09035987 0

[[0.0951133  0.09129983 0.10104895]]
0
[[0.09531471 0.09126253 0.10078421]]
2
[[0.09549607 0.09122893 0.10049856]]
1
[[0.09565938 0.09169635 0.10024135]]
2
[[0.09580643 0.09211724 0.09996269]]
1
[[0.09593885 0.0928262  0.09971177]]
2
[[0.09605808 0.09346459 0.09946246]]
2
[[0.09616544 0.09403943 0.09919123]]
1
[[0.09626211 0.09457947 0.098947  ]]
1
[[0.09634916 0.09499791 0.09872708]]
2
[[0.09642754 0.0953747  0.09908908]]
2
[[0.09649812 0.09571397 0.09984875]]
1
[[0.09656168 0.09591902 0.10053281]]
2
[[0.09661891 0.09610365 0.10114852]]
2
[[0.09667043 0.09626991 0.10165634]]
2
[[0.09671684 0.09641961 0.10206678]]
2
[[0.09675862 0.09655441 0.10238932]]
2
[[0.09679624 0.09667579 0.10263257]]
2
[[0.09683012 0.09678509 0.10280429]]
2
[[0.09686062 0.09688351 0.10291151]]
2
[[0.0968881  0.09697212 0.10296059]]
2
[[0.09691283 0.09705192 0.10295729]]
2
[[0.0969351  0.09712377 0.10290681]]
2
[[0.09695516 0.09718847 0.10281385]]
2
[[0.09697322 0.09724673 0.10268266]]
2
[[0.09698948 0.09729919 0

[[0.09799083 0.10275912 0.10033035]]
2
[[0.098022   0.10290789 0.10010068]]
2
[[0.09805007 0.10304184 0.09987357]]
0
[[0.09803625 0.10316246 0.09966905]]
2
[[0.09802382 0.10327107 0.09947042]]
2
[[0.09801262 0.10336888 0.0992797 ]]
1
[[0.09800253 0.10290284 0.09910797]]
2
Buy: Rs.0.51
[[0.09799345 0.10248319 0.09902871]]
1
[[0.09798528 0.1015704  0.09895735]]
1
[[0.09797791 0.10023163 0.0988931 ]]
2
[[0.09797128 0.09902613 0.09936354]]
2
[[0.09796531 0.09794064 0.10018931]]
1
[[0.09795994 0.09668073 0.10093287]]
2
[[0.09795509 0.09554625 0.10160202]]
2
[[0.09795073 0.0945247  0.10216107]]
2
[[0.09794681 0.09360484 0.10262075]]
2
[[0.09794328 0.09277656 0.10299075]]
2
[[0.09794009 0.09203073 0.10327981]]
2
[[0.09793723 0.09135915 0.10349587]]
2
[[0.09793465 0.09075443 0.10364608]]
2
[[0.09793232 0.0902099  0.10373692]]
2
[[0.09793023 0.08971959 0.10377426]]
2
[[0.09792835 0.08927808 0.10376339]]
2
[[0.09792665 0.08888052 0.10370911]]
2
[[0.09792512 0.08852255 0.10361574]]
2
[[0.09792374

[[0.09210209 0.08354431 0.09106895]]
2
[[0.09209158 0.08335762 0.09076214]]
2
[[0.09208212 0.08318953 0.09045712]]
2
[[0.0920736  0.08303817 0.0901563 ]]
0
[[0.09116524 0.08290188 0.08988543]]
2
[[0.09034731 0.08277915 0.08961271]]
2
[[0.0896108  0.08266865 0.08933387]]
1
[[0.08894761 0.08278592 0.08908279]]
2
[[0.08835044 0.08289152 0.08890557]]
1
[[0.08781271 0.08312452 0.08874598]]
2
[[0.08732852 0.08333433 0.08856319]]
1
[[0.08689253 0.08359369 0.08839859]]
2
[[0.08649994 0.08382723 0.08847585]]
1
[[0.08614644 0.08405734 0.08854541]]
2
[[0.08582813 0.08426454 0.08856905]]
2
[[0.0855415  0.08445111 0.08855132]]
2
[[0.08528341 0.0846191  0.08849632]]
2
[[0.08505101 0.08477038 0.08840778]]
2
[[0.08484174 0.08490659 0.08828907]]
2
[[0.08465331 0.08502924 0.08814321]]
2
[[0.08448364 0.08513969 0.08797298]]
2
[[0.08433086 0.08523914 0.08778084]]
2
[[0.08419328 0.08532869 0.08756906]]
2
[[0.0840694  0.08540933 0.08733966]]
2
[[0.08395786 0.08548193 0.08709449]]
2
[[0.08385742 0.08554731 0

### 4.0 Model Evaluation

In [ ]:
stock_name = input("Enter Stock_name, Model_name")
model_name = input()
model = load_model(model_name)
window_size = model.layers[0].input.shape.as_list()[1]
agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
print(data)
l = len(data) - 1
batch_size = 32
state = getState(data, 0, window_size + 1)
print(state)
total_profit = 0
agent.inventory = []
print(l)
for t in range(l):
    action = agent.act(state)
    print(action)
    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0
    if action == 1: # buy
        agent.inventory.append(data[t])
        print("Buy: " + formatPrice(data[t]))
    elif action == 2 and len(agent.inventory) > 0: # sell
        bought_price = agent.inventory.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += data[t] - bought_price
        print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state
    if done:
        print("--------------------------------")
        print(stock_name + " Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")
        print ("Total profit is:",formatPrice(total_profit))